### learning-AI101 : topic4_2

#### RNN 구현 : 작곡 AI

(복습) 인공지능 구축하는 법
- 0) 데이터 전처리, 데이터 준비하기
  1) 모델링
  2) compile
  3) fit

#### 0. 데이터 전처리, 데이터 준비하기

In [24]:
# 텍스트 파일 열기
data = open('../data/pianoabc/pianoabc.txt', 'r').read()
print(data[:100])

e"G"d2B"D"A3/2B/2c"G"B2G"D"A2e"G"d2B"D"A3/2B/2c:2/4"F"BF:3/4"G"G2e:"C"g2e"Bb"f2d"F"c2AF2e"C"g2e"Bb"f


**위에 보니깐 코드랑 숫자 등이 있음 <-- 느낌 상 abc notation 악보일 가능성이 높음**  
- compile, fit하기 위해서 문자나 숫자로 치환하여야 함
- 악보에 abc notation을 적용하여 음악을 숫자나 문자로 치환하였음
- 숫자는 음표의 길이 / 영어는 CDEFGABC (도레미파솔라시)
- pianoabc.txt는 abc notation으로 짜여진 클래식 악보 몇개를 이어붙인 파일이다
- 따옴표 : 주석

In [25]:
# 문자로 되어있으니 숫자로 바꾸어야 한다 (학습하려면) -> bag of words 만들기
# bag of words : 특정 파일에서 문자가 무엇무엇이 있는지, 숫자가 무엇무엇이 있는지
# a는 1, b는 2... 이런 식으로 문자를 숫자로 치환

bag_of_words = list(set(data)) # set을 이용하여 bag of words 만들기
bag_of_words.sort() # 뒤죽박죽인 set을 위해 sort()처리를 한번 함
print (bag_of_words)
print (len(bag_of_words))

['"', "'", ',', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'z']
31


In [34]:
# bag of words를 이용해서 data_table을 만들어보자

digital_data = data

data_table = {}
numbering = 0

for i in bag_of_words :
    data_table[bag_of_words[numbering]] = numbering
    numbering += 1

print (data_table)

{'"': 0, "'": 1, ',': 2, '/': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, ':': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, '^': 21, '_': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'z': 30}


In [35]:
# data를 숫자화하여 digital_data 저장하기
digital_data = []

for i in data : 
    digital_data.append(data_table[i])
    
print (digital_data[0:10])

[27, 0, 20, 0, 26, 5, 15, 0, 17, 0]


결론
- digital_data : data를 수치화함
- bag_of_words : data에 있는 모든 문자와 숫자
- 그러면 trainX, trianY를 어떻게 만드는가?
  - 만약 [27, 0, 20, 0, 26]이라는 데이터가 있을 때
  - RNN의 목적은 그 다음 걸 예측하는 것이기 때문에,
  - trainX가 [27, 0]이라면 trainY는 [20]
  - trainX가 [27, 0, 20, 0]이라면 trainY는 26이다. X 다음 값이 당연히 Y가 된다 

In [124]:
# trainX와 trainY 만들기
# trainX가 25개일때, trainY가 1개이도록 만들기
length = len(digital_data)
print (length)

train_X = []
train_Y = []
num = 0

while True :
    try : 
        train_X.append(digital_data[num : num + 25])
        train_Y.append(digital_data[num + 25])
        num += 25

    except IndexError :
        break

print (len(train_X[0]), len(train_Y))
train_X.pop()
print (np.array(train_X).shape)

292022
25 11680
(11680, 25)


**np.array를 이용해서 shape을 확인해보자**

In [125]:
import numpy as np
import tensorflow as tf

**원핫인코딩을 하여 학습효율을 높이자**

In [127]:
train_X = tf.one_hot(train_X, 31)
train_Y = tf.one_hot(train_Y, 31)
print(train_X[0:2])

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(2, 25, 31), dtype=float32)


----------------

#### 1. 모델링 및 컴파일

In [133]:
model = tf.keras.models.Sequential([
    # LSTM 레이어를 통해 RNN을 구성하자
    tf.keras.layers.LSTM(100, input_shape=(25, 31)),
    # 문자 31개 중 마지막에 올 문자를 예측하는 문자와 같음 -> 따라서 softmax + categorical_crossentropy를 사용
    tf.keras.layers.Dense(31, activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

# LSTM 레이어를 사용할 땐 activation을 중간에 넣을 필요가 없다
# LSTM 레이어를 사용할 땐 epochs를 엄청 많이 돌려야한다

------

#### 2. fit

In [ ]:
model.fit(train_X, train_Y, batch_size=64, epochs=50)